# Generate results for fig 3
We save them to disk to plot with `fig_3_skill_score_plot.ipynb`

In [ ]:
import plotly.graph_objects as go
from scores.categorical import firm
import xarray as xr
import numpy as np
from scores.stats.statistical_tests import diebold_mariano

In [13]:
fcst = xr.open_mfdataset(
    [
        "data/fcst_2020_2021.nc",
        "data/fcst_2021_2022.nc",
        "data/fcst_2022_2023.nc",
    ]
)
fcst = fcst["__xarray_dataarray_variable__"].compute()

obs = xr.open_mfdataset(
    ["data/obs_2020_2021.nc", "data/obs_2021_2022.nc", "data/obs_2022_2023.nc"]
)
obs = obs["__xarray_dataarray_variable__"].compute()

In [14]:
risk_parameter = 0.5
categorical_thresholds = [1, 3]
threshold_weights = [2, 1]
firm_score = firm(
    fcst,
    obs,
    risk_parameter,
    categorical_thresholds,
    threshold_weights,
    threshold_assignment="upper",
    preserve_dims="all",
)
firm_score

<xarray.Dataset>
Dimensions:                (valid_utc_date: 543, district: 90, lead_day: 7)
Coordinates:
  * valid_utc_date         (valid_utc_date) datetime64[ns] 2020-10-01 ... 202...
  * district               (district) object 'NSW_PW001' ... 'WA_PW014'
  * lead_day               (lead_day) int64 0 1 2 3 4 5 6
Data variables:
    firm_score             (district, lead_day, valid_utc_date) float64 0.0 ....
    overforecast_penalty   (district, lead_day, valid_utc_date) float64 0.0 ....
    underforecast_penalty  (district, lead_day, valid_utc_date) float64 0.0 ....

In [15]:
# Calculate benchmark score of never warning
firm_ref = firm(
    fcst * 0,
    obs,
    risk_parameter,
    categorical_thresholds,
    threshold_weights,
    threshold_assignment="upper",
    preserve_dims="all",
)
firm_ref

<xarray.Dataset>
Dimensions:                (valid_utc_date: 543, district: 90, lead_day: 7)
Coordinates:
  * valid_utc_date         (valid_utc_date) datetime64[ns] 2020-10-01 ... 202...
  * district               (district) object 'NSW_PW001' ... 'WA_PW014'
  * lead_day               (lead_day) int64 0 1 2 3 4 5 6
Data variables:
    firm_score             (district, lead_day, valid_utc_date) float64 0.0 ....
    overforecast_penalty   (district, lead_day, valid_utc_date) float64 0.0 ....
    underforecast_penalty  (district, lead_day, valid_utc_date) float64 0.0 ....

In [16]:
# Calculate if the difference between the forecast FIRM score and the reference FIRM score
# is statistically significant based on the Diebold Mariano test statistic

statistically_different_list = []
for district in firm_score.district:
    district = district.item()
    diff = firm_score.firm_score.sel(district=district) - firm_ref.firm_score.sel(
        district=district
    )
    diff = diff.assign_coords(
        h=(
            "lead_day",
            [
                1,
                2,
                3,
                4,
                5,
                6,
                7,
            ],
        )
    )
    dm_results = diebold_mariano(diff, "lead_day", "h", confidence_level=0.9)

    a = dm_results.ci_upper > 0
    b = dm_results.ci_lower > 0
    c = dm_results.ci_upper < 0
    d = dm_results.ci_lower < 0

    statistically_different = (a * b) + (c * d)
    statistically_different = statistically_different.assign_coords(district=district)
    statistically_different_list.append(statistically_different)
statistically_different = xr.concat(statistically_different_list, dim="district")

/home/nloveday/scores/src/scores/stats/statistical_tests/diebold_mariano_impl.py:191: RuntimeWarning: divide by zero encountered in divide
  "ci_upper": ([ts_dim], ts_mean * (1 + ci_quantile / test_stats)),
/home/nloveday/scores/src/scores/stats/statistical_tests/diebold_mariano_impl.py:191: RuntimeWarning: invalid value encountered in multiply
  "ci_upper": ([ts_dim], ts_mean * (1 + ci_quantile / test_stats)),
/home/nloveday/scores/src/scores/stats/statistical_tests/diebold_mariano_impl.py:192: RuntimeWarning: divide by zero encountered in divide
  "ci_lower": ([ts_dim], ts_mean * (1 - ci_quantile / test_stats)),
/home/nloveday/scores/src/scores/stats/statistical_tests/diebold_mariano_impl.py:192: RuntimeWarning: invalid value encountered in multiply
  "ci_lower": ([ts_dim], ts_mean * (1 - ci_quantile / test_stats)),
/home/nloveday/scores/src/scores/stats/statistical_tests/diebold_mariano_impl.py:191: RuntimeWarning: divide by zero encountered in divide
  "ci_upper": ([ts_dim], ts_mea

In [17]:
firm_ref = firm_ref.mean("valid_utc_date")
firm_score = firm_score.mean("valid_utc_date")
skill_score = 1 - (firm_score.firm_score / firm_ref.firm_score)

In [18]:
ds = xr.Dataset(
    {"skill_score": skill_score, "statistically_different": statistically_different}
)

In [19]:
ds.to_netcdf("results/skill_score.nc")

In [20]:
dm_results

<xarray.Dataset>
Dimensions:          (lead_day: 7)
Coordinates:
  * lead_day         (lead_day) int64 0 1 2 3 4 5 6
Data variables:
    mean             (lead_day) float64 -0.007366 -0.007366 ... 0.01105 0.01289
    dm_test_stat     (lead_day) float64 -1.608 -1.218 0.6004 ... 1.272 1.509
    timeseries_len   (lead_day) int64 543 543 543 543 543 543 543
    confidence_gt_0  (lead_day) float64 0.05394 0.1117 0.7259 ... 0.8983 0.9344
    ci_upper         (lead_day) float64 0.0001699 0.002584 ... 0.02534 0.02694
    ci_lower         (lead_day) float64 -0.0149 -0.01732 ... -0.003243 -0.001158